In [ ]:
import librosa
import librosa.display
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from enum import Enum


class PatternStyle(Enum):
    """Different styles of breakbeats."""
    JUNGLE = "jungle"
    AMEN = "amen"
    TECHSTEP = "techstep"
    ROLLERS = "rollers"  # Rolling bassline style
    FUNK = "funk"


class BreakbeatGenerator:
    def __init__(self, bpm=165, sr=44100, bars=4, kick_sound=None, snare_sound=None, hihat_sound=None):
        self.bpm = bpm
        self.sr = sr
        self.bars = bars
        # Calculate timing
        self.beat_duration = 60 / bpm
        self.bar_duration = self.beat_duration * 4
        self.total_duration = self.bar_duration * bars
        self.length = int(self.total_duration * sr)
        
        # Initialize drum machine
        self.kick = kick_sound
        self.snare = snare_sound
        self.hihat = hihat_sound
        
        # Pattern templates database
        self._init_templates()
    
    def _init_templates(self):
        """Initialize the pattern templates database."""
        self.templates = {
            PatternStyle.JUNGLE: {
                'name': 'jungle',
                'description': 'Standard jungle pattern with syncopation',
                'kick': [0, 8, 16, 24],
                'snare': [8, 24],
                'hihat': [0, 4, 8, 12, 16, 20, 24, 28],
                'ghost_prob': 0.3,
                'roll_prob': 0.4
            },
            PatternStyle.AMEN: {
                'name': 'amen',
                'description': 'Classic Amen break pattern',
                'kick': [0, 8, 12, 16, 24, 28],
                'snare': [4, 12, 20, 28],
                'hihat': [0, 4, 8, 12, 16, 20, 24, 28],
                'ghost_prob': 0.2,
                'roll_prob': 0.6
            },
            PatternStyle.TECHSTEP: {
                'name': 'techstep',
                'description': 'Fast, mechanical DnB pattern',
                'kick': [0, 4, 8, 12, 16, 20, 24, 28],
                'snare': [8, 16, 24],
                'hihat': list(range(0, 32, 2)),  # Every 16th note
                'ghost_prob': 0.1,
                'roll_prob': 0.2
            },
            PatternStyle.ROLLERS: {
                'name': 'rollers',
                'description': 'Rolling, flowing pattern',
                'kick': [0, 16],
                'snare': [8, 24],
                'hihat': [2, 6, 10, 14, 18, 22, 26, 30],  # Off-beats
                'ghost_prob': 0.5,
                'roll_prob': 0.7
            },
            PatternStyle.FUNK: {
                'name': 'funk',
                'description': 'Funky, syncopated pattern',
                "kick": [0, 4, 12, 16, 24, 28],
                "snare": [8, 16, 24],
                "hihat": [2, 6, 10, 14, 18, 22, 26, 30],
                'ghost_prob': 0.5,
                'roll_prob': 0.7
            }
        }
    
    def _place_hit(self, track, start_sample, sound):
        """Place a drum sound at specific position"""
        end_sample = min(start_sample + len(sound), len(track))
        sound_length = end_sample - start_sample
        track[start_sample:end_sample] += sound[:sound_length]
    
    # ENHANCED PATTERN GENERATION
    def create_pattern(self, style=PatternStyle.JUNGLE, complexity=0.7, 
                      variation=0.3, swing=0.0, humanize=0.1):
        """
        Generate a breakbeat pattern with enhanced control.
        
        Args:
            style (PatternStyle): Style of breakbeat
            complexity (float): Pattern complexity (0-1)
            variation (float): Variation from template (0-1)
            swing (float): Swing timing (0-1)
            humanize (float): Human timing variations (0-1)
        
        Returns:
            numpy.ndarray: Stereo breakbeat audio
        """
        if style not in self.templates:
            raise ValueError(f"Style {style} not available. Choose from: {list(self.templates.keys())}")
        
        template = self.templates[style]
        
        # Create grid
        grid_size = 32 * self.bars
        thirtysecond_samples = int((self.total_duration / grid_size) * self.sr)
        
        # Initialize grids
        kick_grid = np.zeros(grid_size)
        snare_grid = np.zeros(grid_size)
        hihat_grid = np.zeros(grid_size)
        
        # Apply template
        for bar in range(self.bars):
            self._apply_template_to_bar(
                template, bar, kick_grid, snare_grid, hihat_grid, 
                variation, complexity
            )
        
        # Apply swing
        if swing > 0:
            kick_grid = self._apply_swing(kick_grid, swing)
            snare_grid = self._apply_swing(snare_grid, swing)
            hihat_grid = self._apply_swing(hihat_grid, swing)
        
        # Apply humanization (timing variations)
        if humanize > 0:
            kick_grid, snare_grid, hihat_grid = self._humanize_grids(
                kick_grid, snare_grid, hihat_grid, humanize
            )
        
        # Render to audio
        audio = self._render_grids(kick_grid, snare_grid, hihat_grid, thirtysecond_samples)
        return audio
    
    def _apply_template_to_bar(self, template, bar, kick_grid, snare_grid, hihat_grid, 
                              variation, complexity):
        """Apply template pattern to a specific bar."""
        bar_start = bar * 32
        
        # Apply kick pattern
        for pos in template['kick']:
            grid_pos = bar_start + pos
            if grid_pos < len(kick_grid):
                vel = 1.0
                if variation > 0 and np.random.random() < variation:
                    vel *= np.random.uniform(0.7, 1.3)
                kick_grid[grid_pos] = vel
        
        # Apply snare pattern
        for pos in template['snare']:
            grid_pos = bar_start + pos
            if grid_pos < len(snare_grid):
                vel = 1.0
                if variation > 0 and np.random.random() < variation:
                    vel *= np.random.uniform(0.7, 1.3)
                snare_grid[grid_pos] = vel
                
                # Add ghost notes based on template probability
                if complexity > 0 and np.random.random() < template['ghost_prob'] * complexity:
                    ghost_pos = (grid_pos + np.random.choice([-2, -1, 1, 2])) % len(snare_grid)
                    if snare_grid[ghost_pos] == 0:
                        snare_grid[ghost_pos] = np.random.uniform(0.1, 0.3)
        
        # Apply hihat pattern
        for pos in template['hihat']:
            grid_pos = bar_start + pos
            if grid_pos < len(hihat_grid):
                vel = np.random.uniform(0.6, 0.9)  # Natural variation for hihats
                hihat_grid[grid_pos] = vel
        
        # Add rolls based on template probability
        if complexity > 0 and np.random.random() < template['roll_prob'] * complexity:
            self._add_snare_roll(snare_grid, bar_start, complexity)
    
    def _apply_swing(self, grid, swing_factor):
        """Apply swing timing to a grid."""
        if swing_factor == 0:
            return grid
        
        swung_grid = np.zeros_like(grid)
        bars = len(grid) // 32
        
        for bar in range(bars):
            bar_start = bar * 32
            for sixteenth in range(0, 32, 2):  # Process eighth note pairs
                pos1 = bar_start + sixteenth
                pos2 = bar_start + sixteenth + 1
                
                # Keep first note straight
                if pos1 < len(grid):
                    swung_grid[pos1] = grid[pos1]
                
                # Swing second note
                if pos2 < len(grid) and grid[pos2] > 0:
                    swing_offset = int(swing_factor * 2)  # Up to 2 positions
                    new_pos = min(pos2 + swing_offset, bar_start + 31)
                    swung_grid[new_pos] = max(swung_grid[new_pos], grid[pos2])
        
        return swung_grid
    
    def _humanize_grids(self, kick_grid, snare_grid, hihat_grid, humanize_factor):
        """Add human timing variations."""
        # Convert grids to lists of hits with timing variations
        grids = [kick_grid, snare_grid, hihat_grid]
        humanized_grids = []
        
        for grid in grids:
            humanized = np.zeros_like(grid)
            hit_indices = np.where(grid > 0)[0]
            
            for idx in hit_indices:
                # Random timing offset (forward or backward)
                offset = int(np.random.uniform(-humanize_factor * 2, humanize_factor * 2))
                new_idx = max(0, min(len(grid) - 1, idx + offset))
                
                # Random velocity variation
                velocity_variation = np.random.uniform(1 - humanize_factor, 1 + humanize_factor)
                humanized[new_idx] = max(humanized[new_idx], grid[idx] * velocity_variation)
            
            humanized_grids.append(humanized)
        
        return humanized_grids
    
    def _add_snare_roll(self, snare_grid, bar_start, complexity):
        """Add a snare roll to the pattern."""
        roll_length = int(np.random.uniform(3, 8) * complexity)
        roll_start = np.random.randint(bar_start + 10, bar_start + 22)
        
        for i in range(roll_length):
            pos = roll_start + i
            if pos < bar_start + 32 and pos < len(snare_grid):
                # Decaying velocity
                vel = 0.9 * (1 - i / roll_length * 0.7)
                snare_grid[pos] = max(snare_grid[pos], vel)
    
    def _render_grids(self, kick_grid, snare_grid, hihat_grid, thirtysecond_samples):
        """Render grids to audio."""
        kick_track = np.zeros(self.length)
        snare_track = np.zeros(self.length)
        hihat_track = np.zeros(self.length)
        
        for i in range(len(kick_grid)):
            start_sample = i * thirtysecond_samples
            if kick_grid[i] > 0:
                self._place_hit(kick_track, start_sample, self.kick * kick_grid[i])
            if snare_grid[i] > 0:
                self._place_hit(snare_track, start_sample, self.snare * snare_grid[i])
            if hihat_grid[i] > 0:
                self._place_hit(hihat_track, start_sample, self.hihat * hihat_grid[i])
        
        # Mix and add stereo width
        return self._mix_and_stereoize(kick_track, snare_track, hihat_track)
    
    def _mix_and_stereoize(self, kick_track, snare_track, hihat_track):
        """Mix tracks and apply stereo processing."""
        mono = kick_track + snare_track + hihat_track
        
        # Simple stereo widening with delay
        stereo = np.zeros((len(mono), 2))
        delay_samples = int(0.001 * self.sr)
        
        stereo[:, 0] = mono
        stereo[delay_samples:, 1] = mono[:-delay_samples] * 0.9
        
        # Normalize
        # if np.max(np.abs(stereo)) > 0:
        #     stereo = stereo / np.max(np.abs(stereo)) * 0.9
        
        return stereo


    # ===============================================
    
    
    # NEW METHOD - CREATE VARIATION
    def create_variation(self, base_style=PatternStyle.JUNGLE, 
                        variation_type="fills", intensity=0.5):
        """
        Create variations of a base pattern.
        
        Args:
            variation_type: "fills", "syncopated", "minimal", "complex"
            intensity: How strong the variation is (0-1)
        """
        base_complexity = 0.5
        
        if variation_type == "fills":
            # Add more fills and rolls
            return self.create_pattern(
                style=base_style,
                complexity=base_complexity + intensity * 0.3,
                variation=intensity * 0.5,
                swing=0.0,
                humanize=intensity * 0.1
            )
        elif variation_type == "syncopated":
            # More syncopation, off-beat emphasis
            return self.create_pattern(
                style=base_style,
                complexity=base_complexity,
                variation=intensity * 0.8,
                swing=intensity * 0.3,
                humanize=0.05
            )
        elif variation_type == "minimal":
            # Sparse, minimal version
            return self.create_pattern(
                style=base_style,
                complexity=base_complexity * (1 - intensity),
                variation=0.1,
                swing=0.0,
                humanize=0.02
            )
        else:
            # Default to original
            return self.create_pattern(style=base_style, complexity=base_complexity)
    
    
    # NEW METHOD - CREATE RANDOM BREAK
    def create_random_break(self, complexity_range=(0.3, 0.9)):
        """Create a random breakbeat using random template and parameters."""
        style = np.random.choice(list(self.templates.keys()))
        complexity = np.random.uniform(*complexity_range)
        variation = np.random.uniform(0.1, 0.6)
        swing = np.random.uniform(0, 0.4)
        humanize = np.random.uniform(0, 0.15)
        
        print(f"Generating {style.value} break with:")
        print(f"  Complexity: {complexity:.2f}")
        print(f"  Variation: {variation:.2f}")
        print(f"  Swing: {swing:.2f}")
        print(f"  Humanize: {humanize:.2f}")
        
        return self.create_pattern(style, complexity, variation, swing, humanize)

In [ ]:
# Initialize
kick_wav_path = 'sound/Kick_Low_Tone_One_Shot.wav'
snare_wav_path = 'sound/shhhhhh1.wav'
hihat_wav_path = 'sound/pulse.wav'

complexity = 4     #0-10
bpm = 170
# ========================================================

kick = librosa.load(kick_wav_path)[0]
snare = librosa.load(snare_wav_path)[0]
hihat = librosa.load(hihat_wav_path)[0]


generator = BreakbeatGenerator(bpm=bpm, bars=4, kick_sound=kick, snare_sound=snare, hihat_sound=hihat)


# Create specific style
jungle = generator.create_pattern(style=PatternStyle.JUNGLE, complexity=0.0)
amen = generator.create_pattern(style=PatternStyle.AMEN, complexity=0.6)
rollers = generator.create_pattern(style=PatternStyle.ROLLERS, complexity=0.5)
funk = generator.create_pattern(style=PatternStyle.FUNK, complexity=0.5)
techstep = generator.create_pattern(style=PatternStyle.TECHSTEP, complexity=0.6)

# Create variations
fill_variation = generator.create_variation(PatternStyle.JUNGLE, "fills", intensity=0.7)
syncopated_version = generator.create_variation(PatternStyle.FUNK, "syncopated", intensity=0.5)
minimal_version = generator.create_variation(PatternStyle.JUNGLE, "minimal", intensity=0.8)

# Random break
random_break = generator.create_random_break()


Generating jungle break with:
  Complexity: 0.36
  Variation: 0.53
  Swing: 0.24
  Humanize: 0.04


In [ ]:

# output_wav_name = f"py_out/py_jungle_break_bpm_{bpm}_complexity_{complexity}_1.wav"
# sf.write(output_wav_name, jungle_break, generator.sr)

In [3]:
PatternStyle.JUNGLE.value

'jungle'